In [3]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma

In [4]:
# 단계 1: 문서 로드(Load Documents)
loader = TextLoader("data/book_document.txt", encoding='utf-8')
docs = loader.load()

In [5]:
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 1


In [7]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [8]:
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 5


In [9]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [10]:
# 단계 4: DB 생성(Create DB) 및 저장

In [11]:
DB_PATH = "./chroma_db"     # 저장 경로

# DB 생성
db = Chroma.from_documents(
    documents=split_documents, 
    embedding=embeddings,
    persist_directory=DB_PATH,
    collection_name='my_db2'
)

In [12]:
for doc in db.similarity_search("돈에 대한 책"):
    print(doc.page_content)

### 제목 : 작심 3일 파이썬 Python
- 저자 : 황덕창
- 발행처 : 스포트라잇북 20190425
- isbn : 9791187431169
- 청구기호 : abc
- 등록번호 : CLP000178589
- 소장위치 : 두 번째 책장 세 번 째 열


### 제목 : 돈의 속성 (300쇄 리커버에디션)
- 저자 : 김승호
- 발행처 : 스노우폭스북스 20200615
- isbn : 9791188331796
- 청구기호 : 일 001.12-김58ㅁ
- 등록번호 : SBBB000029216
- 소장위치 : 두 번째 책장 세 번째 열


### 제목 : 철학의 쓸모
- 저자 : 로랑스 드빌레르
- 발행처 : 피카(FIKA) 20240820
- isbn : 9791193866146
- 청구기호 : 일 100-로835ㅊ
- 등록번호 : CRX000037713
- 소장위치 : 첫 번째 책장 첫 번째 열
### 제목 : 혼자 공부하는 자바
- 저자 : 신용권
- 발행처 : 한빛미디어 2019
- isbn : 9791169211901
- 청구기호 : 일 005.138-신66ㅎ
- 등록번호 : CLP000204092
- 소장위치 : 세 번째 책장 두 번 째 열


### 제목 : 이것이 자바다
- 저자 : 신용권
- 발행처 : 한빛미디어 2015
- isbn : 9791169212274
- 청구기호 : 일 005.133-신66ㅇ
- 등록번호 : CLP000116210
- 소장위치 : 첫 번째 책장 세 번 째 열


### 제목 : 손에잡히는 파이썬
- 저자 : 문용준
- 발행처 : BJpublic(비제이퍼블릭) 2018
- isbn : 9791186697726
- 청구기호 : 일 005.133-문66ㅅ
- 등록번호 : CLP000178588
- 소장위치 : 세 번째 책장 두 번 째 열
### 제목 : 숫자는 거짓말을 한다
- 저자 : 알베르토 카이로
- 발행처 : 웅진지식하우스 20201013
- isbn : 9788901245591
- 청구기호 : 일 310.1-알

In [13]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = db.as_retriever()

In [14]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("파이썬")

[Document(metadata={'source': 'data/book_document.txt'}, page_content='### 제목 : 모두의 파이썬\n- 저자 : 이승찬\n- 발행처 : 길벗 2017\n- isbn : 9791160505856\n- 청구기호 : 일 005.12-이58ㅁ\n- 등록번호 : CRX000029216\n- 소장위치 : 첫 번째 책장 두 번 째 열\n\n\n### 제목 : 자바 웹을 다루는 기술\n- 저자 : 이병승\n- 발행처 : 길벗 2019\n- isbn : 9791160506822\n- 청구기호 : 일 005.138-이44ㅈ\n- 등록번호 : CLP000197301\n- 소장위치 : 첫 번째 책장 두 번 째 열\n\n\n### 제목 : 이것이 자바 스크립트다\n- 저자 : 이우진\n- 발행처 : 한빛미디어 20220905\n- isbn : 9791169210028\n- 청구기호 : sfsdf\n- 등록번호 : abcde\n- 소장위치 : 두 번째 책장 세 번 째 열'),
 Document(metadata={'source': 'data/book_document.txt'}, page_content='### 제목 : 작심 3일 파이썬 Python\n- 저자 : 황덕창\n- 발행처 : 스포트라잇북 20190425\n- isbn : 9791187431169\n- 청구기호 : abc\n- 등록번호 : CLP000178589\n- 소장위치 : 두 번째 책장 세 번 째 열\n\n\n### 제목 : 돈의 속성 (300쇄 리커버에디션)\n- 저자 : 김승호\n- 발행처 : 스노우폭스북스 20200615\n- isbn : 9791188331796\n- 청구기호 : 일 001.12-김58ㅁ\n- 등록번호 : SBBB000029216\n- 소장위치 : 두 번째 책장 세 번째 열\n\n\n### 제목 : 철학의 쓸모\n- 저자 : 로랑스 드빌레르\n- 발행처 : 피카(FIKA) 20240820\n- isbn : 9791193866146\n- 청구

In [15]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [16]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [17]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "파이썬에 대한 책이 있나요?"
response = chain.invoke(question)

In [19]:
print(response)

네, 파이썬에 대한 책이 있습니다. 예를 들어, "작심 3일 파이썬 Python"과 "모두의 파이썬", "손에잡히는 파이썬" 등이 있습니다.
